### Импортируем библиотеки

In [1]:
import psycopg2
import pandas as pd
import numpy as np
import datetime
from datetime import datetime

### Подключаемся к БД 

In [2]:
conn = psycopg2.connect(host = "158.160.52.106",
                        port = 5432,
                        database = "postgres",
                        user = "student",
                        password = "JvLda93aA"
                        )
cur = conn.cursor() 
cur.execute("select c.visit_dttm, c.client_rk, a.account_rk, a2.application_rk, g.game_flg from msu_analytics.client c left join msu_analytics.account a on c.client_rk = a.client_rk left join msu_analytics.application a2 on a.account_rk = a2.account_rk left join msu_analytics.game g on a2.game_rk = g.game_rk")

data = cur.fetchall()
data[:5]

[(datetime.datetime(2022, 12, 20, 19, 28, 38, 474311), 31, 727, None, None),
 (datetime.datetime(2023, 1, 13, 22, 59, 28, 430319), 846, 438, None, None),
 (datetime.datetime(2022, 10, 6, 23, 47, 50, 4362), 2561, 860, None, None),
 (datetime.datetime(2022, 11, 19, 17, 39, 59, 742660), 2753, 519, 439, 0),
 (datetime.datetime(2022, 11, 19, 17, 39, 59, 742660), 2753, 519, 347, 1)]

### Создаем датафрейм

In [3]:
df = pd.read_sql("select c.visit_dttm, c.client_rk, a.account_rk, a2.application_rk, g.game_flg from msu_analytics.client c left join msu_analytics.account a on c.client_rk = a.client_rk left join msu_analytics.application a2 on a.account_rk = a2.account_rk left join msu_analytics.game g on a2.game_rk = g.game_rk", conn)
df

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,visit_dttm,client_rk,account_rk,application_rk,game_flg
0,2022-12-20 19:28:38.474311,31,727.0,NaN,NaN
1,2023-01-13 22:59:28.430319,846,438.0,NaN,NaN
2,2022-10-06 23:47:50.004362,2561,860.0,NaN,NaN
3,2022-11-19 17:39:59.742660,2753,519.0,439.0,0.0
4,2022-11-19 17:39:59.742660,2753,519.0,347.0,1.0
...,...,...,...,...,...
4310,2022-10-03 01:25:22.239366,3743,692.0,242.0,1.0
4311,2023-01-23 18:53:47.521661,818,597.0,61.0,0.0
4312,2023-01-10 13:03:54.168220,2216,480.0,51.0,1.0
4313,2022-12-20 04:37:44.493718,3057,533.0,11.0,0.0


### Обрабатываем данные 

In [4]:
df.loc[df['game_flg'] == 0, 'game_flg'] = np.NaN
df['date'] = df['visit_dttm'].map(lambda x: x.strftime("%Y-%m"))
df['date2'] = df['visit_dttm'].map(lambda x: x.strftime("%Y-%m-%d"))
df['date2'] = pd.to_datetime(df['date2']) - pd.to_timedelta(7, unit = 'd')
df = df.sort_values(by = ['game_flg'])
df = df.drop_duplicates(subset = ['client_rk'])
# отсортировать по последнему значению и удалить дубликаты по перовму 
df

,visit_dttm,client_rk,account_rk,application_rk,game_flg,date,date2
4,2022-11-19 17:39:59.742660,2753,519.0,347.0,1.0,2022-11,2022-11-12
2022,2022-09-11 20:27:12.284179,2402,578.0,476.0,1.0,2022-09,2022-09-04
2023,2022-10-29 11:20:56.580162,3002,653.0,133.0,1.0,2022-10,2022-10-22
2024,2022-12-12 02:58:06.922762,4079,525.0,249.0,1.0,2022-12,2022-12-05
2025,2022-09-19 05:13:59.671943,3628,541.0,28.0,1.0,2022-09,2022-09-12
...,...,...,...,...,...,...,...
4277,2023-01-21 04:15:53.791273,4087,NaN,NaN,NaN,2023-01,2023-01-14
4279,2022-10-03 10:08:23.663117,2444,601.0,498.0,NaN,2022-10,2022-09-26
4281,2022-12-18 23:01:44.073116,1452,617.0,101.0,NaN,2022-12,2022-12-11
4287,2023-01-03 09:48:37.109344,2430,751.0,182.0,NaN,2023-01,2022-12-27


### Группируем по месяцам 

In [5]:
new_df = df.groupby(df.date)['client_rk', 'account_rk', 'application_rk', 'game_flg'].count()
new_df

<ipython-input-5-8bf79aba6ab0>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  new_df = df.groupby(df.date)['client_rk', 'account_rk', 'application_rk', 'game_flg'].count()


,client_rk,account_rk,application_rk,game_flg
date,,,,
2022-09,778,190,51,39
2022-10,843,217,65,56
2022-11,786,200,56,50
2022-12,805,196,57,33
2023-01,852,213,64,20
2023-02,32,8,0,0


### Группируем по неделям 

In [6]:
new_df2 = df.groupby([pd.Grouper(key = 'date2', freq = 'W')])['client_rk', 'account_rk', 'application_rk', 'game_flg'].count()
new_df2

<ipython-input-6-11b4f0c262ab>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  new_df2 = df.groupby([pd.Grouper(key = 'date2', freq = 'W')])['client_rk', 'account_rk', 'application_rk', 'game_flg'].count()


,client_rk,account_rk,application_rk,game_flg
date2,,,,
2022-08-28,111,32,7,5
2022-09-04,184,31,6,5
2022-09-11,167,43,16,8
2022-09-18,181,45,12,11
2022-09-25,195,59,17,17
2022-10-02,204,58,18,15
2022-10-09,190,44,13,13
2022-10-16,171,41,12,10
2022-10-23,189,48,15,11


### Считаем конверсии 

In [7]:
a1 = new_df['account_rk']*100/new_df['client_rk']
a2 = new_df['application_rk']*100/new_df['client_rk']
a3 = new_df['game_flg']*100/new_df['client_rk']
new_df['account_rk'] = a1
new_df['application_rk'] = a2
new_df['game_flg'] = a3
new_df

,client_rk,account_rk,application_rk,game_flg
date,,,,
2022-09,778,24.421594,6.555270,5.012853
2022-10,843,25.741400,7.710558,6.642942
2022-11,786,25.445293,7.124682,6.361323
2022-12,805,24.347826,7.080745,4.099379
2023-01,852,25.000000,7.511737,2.347418
2023-02,32,25.000000,0.000000,0.000000


In [8]:
b1 = new_df2['account_rk']*100/new_df2['client_rk']
b2 = new_df2['application_rk']*100/new_df2['client_rk']
b3 = new_df2['game_flg']*100/new_df2['client_rk']
new_df2['account_rk'] = b1
new_df2['application_rk'] = b2
new_df2['game_flg'] = b3
new_df2

,client_rk,account_rk,application_rk,game_flg
date2,,,,
2022-08-28,111,28.828829,6.306306,4.504505
2022-09-04,184,16.847826,3.260870,2.717391
2022-09-11,167,25.748503,9.580838,4.790419
2022-09-18,181,24.861878,6.629834,6.077348
2022-09-25,195,30.256410,8.717949,8.717949
2022-10-02,204,28.431373,8.823529,7.352941
2022-10-09,190,23.157895,6.842105,6.842105
2022-10-16,171,23.976608,7.017544,5.847953
2022-10-23,189,25.396825,7.936508,5.820106


### Импортируем библиотеки для визуализации

In [9]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [10]:
date = new_df.index[:-1]
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x = date, y = new_df['account_rk'][:-1], name = 'Из посетителя сайта в зарегистрированного пользователя'))
fig.add_trace(go.Bar(x = date, y = new_df['application_rk'][:-1], name = 'Из посетителя сайта в заявку на игру'))
fig.add_trace(go.Bar(x = date, y = new_df['game_flg'][:-1], name = 'Из посетителя сайта в посетителя квеста'))
fig.add_trace(go.Scatter(y= new_df['client_rk'][:-1], x=date, name = 'Количество посетителей сайта'), secondary_y=True)
fig.update_yaxes(title_text='Конверсия в %', secondary_y=False)
fig.update_yaxes(title_text='Количество посетителей сайта', secondary_y=True)
fig.update_layout(legend_orientation="h", title = 'Воронка продаж по месяцам | Тинькофф Квест')
fig.show()

In [11]:
date = new_df2.index[:-1]
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x = date, y = new_df2['account_rk'][:-1], name = 'Из посетителя сайта в зарегистрированного пользователя'))
fig.add_trace(go.Bar(x = date, y = new_df2['application_rk'][:-1], name = 'Из посетителя сайта в заявку на игру'))
fig.add_trace(go.Bar(x = date, y = new_df2['game_flg'][:-1], name = 'Из посетителя сайта в посетителя квеста'))
fig.add_trace(go.Scatter(y= new_df2['client_rk'][:-1], x=date, name = 'Количество посетителей сайта'), secondary_y=True)
fig.update_yaxes(title_text='Конверсия в %', secondary_y=False)
fig.update_yaxes(title_text='Количество посетителей сайта', secondary_y=True)
fig.update_layout(legend_orientation="h", title = 'Воронка продаж по неделям | Тинькофф Квест')
fig.show()

### Выводы
Видим падение конверсии из посетителей сайта в тех, кто по факту приходит на игру до 2.4 % в январе.
#### Гипотезы о причинах снижения конверсии: 
- проблемы с уведомлениями клиентов о предстоящей игре; 
- фактор сезонности (возможно, перед новогодними праздниками и после них клиенты забывают о том, что зарегистрировалимсь на квест). Данный фактор нельзя проверить, так как недостаточно данных.
 
#### Точки роста:
- обзвон клиентов за день до игры для повышения конверсии из посетителей сайта в тех кто по факту приходит на игру;
- приветственный бонус (скидка на первую игру или бонусы в течение какого-то времени за регистрацию на сайте), данный маркетинговый прием поможет повысить сразу две конверсии: из посетителей сайта в регистрацию, благодаря скидки или бонусам, и из посетителя сайта в заявку, потому что бонусами или скидкой необходимо воспользоваться за какой-то промежуток времени. 